In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV



In [10]:
test_df = pd.read_csv('../datasets/Train_Test_data/NLP_features_Test.csv')
train_df = pd.read_csv('../datasets/Train_Test_data/NLP_features_Train.csv')

In [22]:

# Ensure TF-IDF columns are arrays
train_df['TF-IDF'] = train_df['TF-IDF'].apply(lambda x: x if isinstance(x, (list, np.ndarray)) else [0]*300)
test_df['TF-IDF'] = test_df['TF-IDF'].apply(lambda x: x if isinstance(x, (list, np.ndarray)) else [0]*300)

# Convert to NumPy arrays
X_train = np.vstack(train_df['TF-IDF'].values)
X_test = np.vstack(test_df['TF-IDF'].values)

# Labels
y_train = train_df['Emotion']
y_test = test_df['Emotion']


---

# First itteration of Naive Bayes and Multinomial Naive Bayes

In [12]:
# ----------------------
# Multinomial Naive Bayes
# ----------------------
mnb = MultinomialNB()
y_pred_mnb = mnb.fit(X_train, y_train).predict(X_test)
print("Multinomial Naive Bayes Report:")
print(classification_report(y_test, y_pred_mnb, target_names=np.unique(y_train)))

Multinomial Naive Bayes Report:
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        46
     disgust       0.00      0.00      0.00         2
        fear       0.00      0.00      0.00        17
   happiness       0.00      0.00      0.00       243
     neutral       0.33      1.00      0.49       237
     sadness       0.00      0.00      0.00        30
    surprise       0.00      0.00      0.00       147

    accuracy                           0.33       722
   macro avg       0.05      0.14      0.07       722
weighted avg       0.11      0.33      0.16       722



c:\Users\Beheerder\anaconda3\envs\block_c_Y2\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Beheerder\anaconda3\envs\block_c_Y2\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Beheerder\anaconda3\envs\block_c_Y2\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()

---

# Second itteration with Gridsearch

In [21]:
pipeline = Pipeline([
    ('select', SelectKBest(chi2, k=5000)),
    ('clf', MultinomialNB())
])

params = {
    'select__k': ['all'],
    'clf__alpha': [0.1, 0.5, 1.0]
}

grid = GridSearchCV(pipeline, params, cv=5, scoring='f1_macro')
grid.fit(X_train, y_train)
print("Best params:", grid.best_params_)

y_pred = grid.predict(X_test)
print(classification_report(y_test, y_pred))


Best params: {'clf__alpha': 0.1, 'select__k': 'all'}
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        46
     disgust       0.00      0.00      0.00         2
        fear       0.00      0.00      0.00        17
   happiness       0.00      0.00      0.00       243
     neutral       0.33      1.00      0.49       237
     sadness       0.00      0.00      0.00        30
    surprise       0.00      0.00      0.00       147

    accuracy                           0.33       722
   macro avg       0.05      0.14      0.07       722
weighted avg       0.11      0.33      0.16       722



c:\Users\Beheerder\anaconda3\envs\block_c_Y2\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Beheerder\anaconda3\envs\block_c_Y2\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Beheerder\anaconda3\envs\block_c_Y2\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()